In [27]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [38]:
root_dir="C:\College stuff'\Sem4\Mini Project\Main2"
number_of_images={}

for dir in os.listdir(root_dir):
    number_of_images[dir]=len(os.listdir(os.path.join(root_dir,dir)))

In [39]:
number_of_images

{'Healthy': 3494, 'Tumor': 6517}

We will split the data 
70% for training 
15% for validation
15% for testing 

We will create a training folder

In [40]:
def datafolder(path,split):
    if not os.path.exists("./"+path):
      os.mkdir("./"+path)

      for dir in os.listdir(root_dir):
        os.makedirs("./"+path+"/"+dir)

        for img in np.random.choice(a=os.listdir(os.path.join(root_dir,dir)),size=(math.floor(split*number_of_images[dir])-2),replace=False):
            o=os.path.join(root_dir,dir,img)
            d=os.path.join("./"+path,dir)
            shutil.copy(o,d)
            os.remove(o)  
    else:
         print("the folder exists")



In [41]:
datafolder("Train",0.7)

In [42]:
datafolder("Test",0.15)


In [43]:
datafolder("Val",0.15)

Model Building

In [44]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAveragePooling2D
from keras.models import Sequential
from tensorflow.keras.utils import load_img
import keras
from keras.preprocessing.image import ImageDataGenerator


CNN Model

In [45]:
model=Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))

model.add(Conv2D(filters=366, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1,activation='relu'))

model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 366)     53070     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 366)    0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      210880    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

In [47]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

Preparing our data generator

In [48]:
def preprocessingimages(path):
    image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255, horizontal_flip=True)
    image=image_data.flow_from_directory(directory=path, target_size=(224,224), batch_size=32, class_mode='binary')
    return image

In [49]:
path="C:\College stuff'\Sem4\Mini Project\Train"
train_data=preprocessingimages(path)

Found 7002 images belonging to 2 classes.


In [50]:
def preprocessingimages2(path):
    image_data=ImageDataGenerator(rescale=1/255)
    image=image_data.flow_from_directory(directory=path, target_size=(224,224), batch_size=32, class_mode='binary')
    return image

In [51]:
path="C:\College stuff'\Sem4\Mini Project\Test"
test_data=preprocessingimages2(path)

Found 1497 images belonging to 2 classes.


In [52]:
path="C:\College stuff'\Sem4\Mini Project\Val"
val_data=preprocessingimages2(path)

Found 1497 images belonging to 2 classes.


Early Stopping and model checkpoint


In [53]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Early Stopping
ec=EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=3, verbose=1, mode='auto')

#model checpoint
mc=ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel.h5", verbose=1, save_best_only=True, mode='auto')

cd=[ec,mc]

Model Training

In [54]:
hs=model.fit_generator(generator=train_data,steps_per_epoch=8,epochs=30,verbose=1, validation_data=val_data, validation_steps=16, callbacks=cd)

C:\Users\Paras Shirish Vaidya\AppData\Local\Temp\ipykernel_336\2745458043.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hs=model.fit_generator(generator=train_data,steps_per_epoch=8,epochs=30,verbose=1, validation_data=val_data, validation_steps=16, callbacks=cd)


Epoch 1/30
8/8 [==============================] - ETA: 0s - loss: 6.3917 - accuracy: 0.5625 
Epoch 1: val_accuracy improved from -inf to 0.65430, saving model to .\bestmodel.h5
8/8 [==============================] - 206s 25s/step - loss: 6.3917 - accuracy: 0.5625 - val_loss: 5.2717 - val_accuracy: 0.6543
Epoch 2/30
8/8 [==============================] - ETA: 0s - loss: 5.7185 - accuracy: 0.6250 
Epoch 2: val_accuracy improved from 0.65430 to 0.65820, saving model to .\bestmodel.h5
8/8 [==============================] - 185s 23s/step - loss: 5.7185 - accuracy: 0.6250 - val_loss: 5.2121 - val_accuracy: 0.6582
Epoch 3/30
8/8 [==============================] - ETA: 0s - loss: 5.0037 - accuracy: 0.6719 
Epoch 3: val_accuracy did not improve from 0.65820
8/8 [==============================] - 178s 23s/step - loss: 5.0037 - accuracy: 0.6719 - val_loss: 5.2717 - val_accuracy: 0.6543
Epoch 4/30
1/8 [==>...........................] - ETA: 2:01 - loss: 5.7185 - accuracy: 0.6250